In [1]:
import numpy as np
import torch

from ssd.utils_ssd300 import dboxes300_coco
from ssd.create_model import nvidia_ssd
from ssd.utils_ssd300 import Encoder
from ssd.model import Loss, SSD300
from ssd.SSD_Transformers import SSDTransformer
from utils.utils import set_seed
from train_model import train_model
from ssd.model_eval import model_evaluate


from torch.utils.data import DataLoader

from config.config import Configs
from ssd.model_eval import model_evaluate

from ssd.decode_results import Processing as processing
from ssd.loader_crowdhuman import CrowdHuman


c:\Users\123\Desktop\torch cuda\.venv\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
c:\Users\123\Desktop\torch cuda\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda'
ssd_300 = nvidia_ssd(device=device, pretrainded_custom=False, pretrained_default=True)

feat_ext_w = ssd_300.feature_extractor.state_dict()

model = SSD300(label_num=3)
model = model.cuda()

model.feature_extractor.load_state_dict(feat_ext_w)


<All keys matched successfully>

In [3]:
batch_size = 32
device = "cuda"

for param in model.parameters():
    param.requires_grad = True

for param in model.feature_extractor.parameters():
    param.requires_grad = False

dboxes = dboxes300_coco()
encoder = Encoder(dboxes)

loss_func = Loss(dboxes, device=device)
transformers_ssd_train = SSDTransformer(dboxes, size=(300, 300), val=False)
transformers_ssd_val = SSDTransformer(dboxes, size=(300, 300), val=True)
set_seed()
data_train = CrowdHuman(
    img_folder="data\\train\\",
    annotate_file="data\\annotation_train.json",
    transform=transformers_ssd_train,
    use_head_bbx=True,
)


train_dataloader = DataLoader(
    data_train,
    batch_size=batch_size,
    shuffle=True 
)


data_val = CrowdHuman(
    img_folder="data\\val\\",
    annotate_file="data\\annotation_val.json",
    transform=transformers_ssd_val,
    use_head_bbx=True,
)

val_dataloader = DataLoader(
    data_val,
    batch_size=batch_size,
    shuffle=False,  
)


data_test = CrowdHuman(
    img_folder="data\\test\\",
    annotate_file="data\\annotation_test.json",
    transform=transformers_ssd_val,
    use_head_bbx=True,
)

test_dataloader = DataLoader(
    data_test,
    batch_size=batch_size,
    shuffle=False,  
)


# https://www.kaggle.com/code/billiemage/understand-lr-scheduler-with-simple-examples
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.001)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, patience=4,threshold = 1e-3, verbose=True, factor=0.5, mode = 'max'
)

load annotation file:  data\annotation_train.json
load annotation file:  data\annotation_val.json
load annotation file:  data\annotation_test.json


In [20]:
checkpoint = torch.load('weight\state best_model_at_19_200.pth')

In [4]:
checkpoint.keys()

NameError: name 'checkpoint' is not defined

In [11]:
checkpoint = torch.load('weight\\state best_model_at_adam2.pth')
model.load_state_dict(checkpoint['model'])
#optimizer.load_state_dict(checkpoint['optimizer_state'])
#scheduler.load_state_dict(checkpoint['lr_scheduler_state'])
for param in model.parameters():
    param.requires_grad = True

for param in model.feature_extractor.parameters():
    param.requires_grad = False

In [12]:
res2 = model_evaluate(
    model = model,
    encoder = encoder,
    val_dataloader = test_dataloader,
    device=device,
    use_pick_best=False,
    threshold = Configs.decode_result["pic_threshold"]
)

Evaluating


100%|██████████| 47/47 [04:09<00:00,  5.31s/it]


In [14]:
res2

{'map': tensor(0.1682),
 'map_50': tensor(0.3849),
 'map_75': tensor(0.1273),
 'map_small': tensor(0.0188),
 'map_medium': tensor(0.1921),
 'map_large': tensor(0.3968),
 'mar_1': tensor(0.0249),
 'mar_10': tensor(0.1502),
 'mar_100': tensor(0.2364),
 'mar_small': tensor(0.0603),
 'mar_medium': tensor(0.2761),
 'mar_large': tensor(0.4846),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.),
 'classes': tensor([1, 2], dtype=torch.int32)}

In [17]:
res2

{'map': tensor(8.3266e-06),
 'map_50': tensor(4.9525e-05),
 'map_75': tensor(8.8619e-07),
 'map_small': tensor(0.0001),
 'map_medium': tensor(1.3026e-05),
 'map_large': tensor(1.2189e-05),
 'mar_1': tensor(1.4826e-05),
 'mar_10': tensor(0.0002),
 'mar_100': tensor(0.0015),
 'mar_small': tensor(4.2412e-05),
 'mar_medium': tensor(0.0034),
 'mar_large': tensor(0.0035),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.),
 'classes': tensor([1, 2], dtype=torch.int32)}

In [9]:
res1

{'map': tensor(0.1473),
 'map_50': tensor(0.3136),
 'map_75': tensor(0.1213),
 'map_small': tensor(0.0141),
 'map_medium': tensor(0.1637),
 'map_large': tensor(0.3634),
 'mar_1': tensor(0.0248),
 'mar_10': tensor(0.1440),
 'mar_100': tensor(0.1817),
 'mar_small': tensor(0.0252),
 'mar_medium': tensor(0.2084),
 'mar_large': tensor(0.4266),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.),
 'classes': tensor([1, 2], dtype=torch.int32)}

In [5]:
res = model_evaluate(
    model = model,
    encoder = encoder,
    val_dataloader = test_dataloader,
    device=device,
    use_pick_best=False,
    threshold = Configs.decode_result["pic_threshold"]
)

Evaluating


100%|██████████| 47/47 [04:10<00:00,  5.33s/it]
c:\Users\123\Desktop\torch cuda\.venv\lib\site-packages\torchmetrics\utilities\prints.py:42: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)  # noqa: B028


In [6]:
res

{'map': tensor(0.1682),
 'map_50': tensor(0.3849),
 'map_75': tensor(0.1273),
 'map_small': tensor(0.0188),
 'map_medium': tensor(0.1921),
 'map_large': tensor(0.3968),
 'mar_1': tensor(0.0249),
 'mar_10': tensor(0.1502),
 'mar_100': tensor(0.2364),
 'mar_small': tensor(0.0603),
 'mar_medium': tensor(0.2761),
 'mar_large': tensor(0.4846),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.),
 'classes': tensor([1, 2], dtype=torch.int32)}

In [8]:
checkpoint = torch.load('weight\state best_model_at_adam1.pth')
model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer_state'])
#scheduler.load_state_dict(checkpoint['lr_scheduler_state'])
for param in model.parameters():
    param.requires_grad = True

for param in model.feature_extractor.parameters():
    param.requires_grad = False

In [19]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [23]:
train_model(
    model=model,
    encoder = encoder,
    num_epoch=100,
    optimizer=optimizer,
    loss_func = loss_func,
    train_dataloader = train_dataloader,
    val_dataloader = val_dataloader,
    device  = device,
    scheduler=scheduler,
    path_to_save=Configs.path_to_save_state_model,
    use_pick_best=Configs.use_pick_best_in_eval
)


EPOCH 1 of 100
Training


100%|██████████| 235/235 [08:05<00:00,  2.07s/it]


Evaluating


100%|██████████| 47/47 [04:04<00:00,  5.20s/it]


Epoch #1 train loss: 4.816
Epoch #1 mAP: 0.12595567107200623
Epoch #1 mAP_50: 0.3397454619407654
Epoch #1 best mAP: 0.12595567107200623 at epoch = 1
Took 13.181 minutes for epoch 1

EPOCH 2 of 100
Training


...

Epoch #99 train loss: 3.526
Epoch #99 mAP: 0.16209669411182404
Epoch #99 mAP_50: 0.37306541204452515
Epoch #99 best mAP: 0.1622755080461502 at epoch = 83
Took 13.568 minutes for epoch 99

EPOCH 100 of 100
Training


100%|██████████| 235/235 [08:37<00:00,  2.20s/it]


Evaluating


100%|██████████| 47/47 [04:13<00:00,  5.39s/it]


Epoch #100 train loss: 3.518
Epoch #100 mAP: 0.16215237975120544
Epoch #100 mAP_50: 0.373604953289032
Epoch #100 best mAP: 0.1622755080461502 at epoch = 83
Took 13.834 minutes for epoch 100


In [ ]:
checkpoint = torch.load('weight\state best_model_at_adam1.pth')
model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer_state'])
scheduler.load_state_dict(checkpoint['lr_scheduler_state'])
for param in model.parameters():
    param.requires_grad = True

for param in model.feature_extractor.parameters():
    param.requires_grad = False

In [14]:
x = model_evaluate(
    model = model,
    encoder = encoder,
    val_dataloader = val_dataloader,
    device=device,
    use_pick_best=False,
    threshold = Configs.decode_result["pic_threshold"]
)

Evaluating


100%|██████████| 47/47 [04:14<00:00,  5.42s/it]


In [15]:
x

{'map': tensor(0.1314),
 'map_50': tensor(0.3248),
 'map_75': tensor(0.0852),
 'map_small': tensor(0.0134),
 'map_medium': tensor(0.1513),
 'map_large': tensor(0.3177),
 'mar_1': tensor(0.0222),
 'mar_10': tensor(0.1280),
 'mar_100': tensor(0.1951),
 'mar_small': tensor(0.0449),
 'mar_medium': tensor(0.2280),
 'mar_large': tensor(0.4136),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.),
 'classes': tensor([1, 2], dtype=torch.int32)}

In [ ]:

checkpoint = torch.load('weight\state best_model_at_19_monin.pth')
model.load_state_dict(checkpoint['model_state'])
optimizer.load_state_dict(checkpoint['optimizer_state'])
scheduler.load_state_dict(checkpoint['lr_scheduler_state'])
for param in model.parameters():
    param.requires_grad = True

for param in model.feature_extractor.parameters():
    param.requires_grad = False